In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required Packages
!pip install torch numpy transformers datasets matplotlib tqdm scikit-learn openai dotmap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import datasets
import transformers
import re
import torch
import torch.nn.functional as F
import tqdm
import random
from sklearn.metrics import roc_curve, precision_recall_curve, auc,  f1_score, accuracy_score, confusion_matrix, precision_score, recall_score
import argparse
import datetime
import os
import json
import functools
from multiprocessing.pool import ThreadPool
import time
import pandas as pd
import subprocess
from dotmap import DotMap
import seaborn as sns


# 15 colorblind-friendly colors
COLORS = ["#0072B2", "#009E73", "#D55E00", "#CC79A7", "#F0E442",
            "#56B4E9", "#E69F00", "#000000", "#0072B2", "#009E73",
            "#D55E00", "#CC79A7", "#F0E442", "#56B4E9", "#E69F00"]

# define regex to match all <extra_id_*> tokens, where * is an integer
pattern = re.compile(r"<extra_id_\d+>")


def load_base_model():
    print('MOVING BASE MODEL TO GPU...', end='', flush=True)
    start = time.time()
    try:
        mask_model.cpu()
    except NameError:
        pass
    if args.openai_model is None:
        base_model.to(DEVICE)
    print(f'DONE ({time.time() - start:.2f}s)')


def load_mask_model():
    print('MOVING MASK MODEL TO GPU...', end='', flush=True)
    start = time.time()

    if args.openai_model is None:
        base_model.cpu()
    if not args.random_fills:
        mask_model.to(DEVICE)
    print(f'DONE ({time.time() - start:.2f}s)')


def tokenize_and_mask(text, span_length, pct, ceil_pct=False):
    tokens = text.split(' ')
    mask_string = '<<<mask>>>'

    n_spans = pct * len(tokens) / (span_length + args.buffer_size * 2)
    if ceil_pct:
        n_spans = np.ceil(n_spans)
    n_spans = int(n_spans)

    n_masks = 0
    while n_masks < n_spans:
        start = np.random.randint(0, len(tokens) - span_length)
        end = start + span_length
        search_start = max(0, start - args.buffer_size)
        search_end = min(len(tokens), end + args.buffer_size)
        if mask_string not in tokens[search_start:search_end]:
            tokens[start:end] = [mask_string]
            n_masks += 1

    # replace each occurrence of mask_string with <extra_id_NUM>, where NUM increments
    num_filled = 0
    for idx, token in enumerate(tokens):
        if token == mask_string:
            tokens[idx] = f'<extra_id_{num_filled}>'
            num_filled += 1
    assert num_filled == n_masks, f"num_filled {num_filled} != n_masks {n_masks}"
    text = ' '.join(tokens)
    return text


def count_masks(texts):
    return [len([x for x in text.split() if x.startswith("<extra_id_")]) for text in texts]


# replace each masked span with a sample from T5 mask_model
def replace_masks(texts):
    n_expected = count_masks(texts)
    stop_id = mask_tokenizer.encode(f"<extra_id_{max(n_expected)}>")[0]
    tokens = mask_tokenizer(texts, return_tensors="pt", padding=True).to(DEVICE)
    outputs = mask_model.generate(**tokens, max_length=150, do_sample=True, top_p=args.mask_top_p, num_return_sequences=1, eos_token_id=stop_id)
    return mask_tokenizer.batch_decode(outputs, skip_special_tokens=False)


def extract_fills(texts):
    # remove <pad> from beginning of each text
    texts = [x.replace("<pad>", "").replace("</s>", "").strip() for x in texts]

    # return the text in between each matched mask token
    extracted_fills = [pattern.split(x)[1:-1] for x in texts]

    # remove whitespace around each fill
    extracted_fills = [[y.strip() for y in x] for x in extracted_fills]

    return extracted_fills


def apply_extracted_fills(masked_texts, extracted_fills):
    # split masked text into tokens, only splitting on spaces (not newlines)
    tokens = [x.split(' ') for x in masked_texts]

    n_expected = count_masks(masked_texts)

    # replace each mask token with the corresponding fill
    for idx, (text, fills, n) in enumerate(zip(tokens, extracted_fills, n_expected)):
        if len(fills) < n:
            tokens[idx] = text
        else:
            for fill_idx in range(n):
                text[text.index(f"<extra_id_{fill_idx}>")] = fills[fill_idx]

    # join tokens back into text
    texts = [" ".join(x) for x in tokens]
    return texts


def perturb_texts_(texts, span_length, pct, ceil_pct=False):
    if not args.random_fills:
        masked_texts = [tokenize_and_mask(x, span_length, pct, ceil_pct) for x in texts]

        raw_fills = replace_masks(masked_texts)
        extracted_fills = extract_fills(raw_fills)
        perturbed_texts = apply_extracted_fills(masked_texts, extracted_fills)


        # Handle the fact that sometimes the model doesn't generate the right number of fills and we have to try again
        attempts = 1
        while '' in perturbed_texts:
            #print(masked_texts)
            #print(extracted_fills)
            #print(perturbed_texts)
            idxs = [idx for idx, x in enumerate(perturbed_texts) if x == '']
            #print(idxs)
            print(f'WARNING: {len(idxs)} texts have no fills. Trying again [attempt {attempts}].')
            masked_texts = [tokenize_and_mask(x, span_length, pct, ceil_pct) for idx, x in enumerate(texts) if idx in idxs]
            raw_fills = replace_masks(masked_texts)
            extracted_fills = extract_fills(raw_fills)
            new_perturbed_texts = apply_extracted_fills(masked_texts, extracted_fills)
            for idx, x in zip(idxs, new_perturbed_texts):
                perturbed_texts[idx] = x
            attempts += 1
            if attempts == 10:
                for idx, x in zip(idxs, new_perturbed_texts):
                    perturbed_texts[idx] = 'hello'
                break
    else:
        if args.random_fills_tokens:
            # tokenize base_tokenizer
            tokens = base_tokenizer(texts, return_tensors="pt", padding=True).to(DEVICE)
            valid_tokens = tokens.input_ids != base_tokenizer.pad_token_id
            replace_pct = args.pct_words_masked * (args.span_length / (args.span_length + 2 * args.buffer_size))

            # replace replace_pct of input_ids with random tokens
            random_mask = torch.rand(tokens.input_ids.shape, device=DEVICE) < replace_pct
            random_mask &= valid_tokens
            random_tokens = torch.randint(0, base_tokenizer.vocab_size, (random_mask.sum(),), device=DEVICE)
            # while any of the random tokens are special tokens, replace them with random non-special tokens
            while any(base_tokenizer.decode(x) in base_tokenizer.all_special_tokens for x in random_tokens):
                random_tokens = torch.randint(0, base_tokenizer.vocab_size, (random_mask.sum(),), device=DEVICE)
            tokens.input_ids[random_mask] = random_tokens
            perturbed_texts = base_tokenizer.batch_decode(tokens.input_ids, skip_special_tokens=True)
        else:
            masked_texts = [tokenize_and_mask(x, span_length, pct, ceil_pct) for x in texts]
            perturbed_texts = masked_texts
            # replace each <extra_id_*> with args.span_length random words from FILL_DICTIONARY
            for idx, text in enumerate(perturbed_texts):
                filled_text = text
                for fill_idx in range(count_masks([text])[0]):
                    fill = random.sample(FILL_DICTIONARY, span_length)
                    filled_text = filled_text.replace(f"<extra_id_{fill_idx}>", " ".join(fill))
                assert count_masks([filled_text])[0] == 0, "Failed to replace all masks"
                perturbed_texts[idx] = filled_text

    return perturbed_texts


def perturb_texts(texts, span_length, pct, ceil_pct=False):
    chunk_size = args.chunk_size
    if '11b' in mask_filling_model_name:
        chunk_size //= 2

    outputs = []
    for i in tqdm.tqdm(range(0, len(texts), chunk_size), desc="Applying perturbations"):
        outputs.extend(perturb_texts_(texts[i:i + chunk_size], span_length, pct, ceil_pct=ceil_pct))
    return outputs


def drop_last_word(text):
    return ' '.join(text.split(' ')[:-1])


def get_likelihood(logits, labels):
    assert logits.shape[0] == 1
    assert labels.shape[0] == 1

    logits = logits.view(-1, logits.shape[-1])[:-1]
    labels = labels.view(-1)[1:]
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    log_likelihood = log_probs.gather(dim=-1, index=labels.unsqueeze(-1)).squeeze(-1)
    return log_likelihood.mean()


# Get the log likelihood of each text under the base_model
def get_ll(text):
    if args.openai_model:
        kwargs = { "engine": args.openai_model, "temperature": 0, "max_tokens": 0, "echo": True, "logprobs": 0}
        r = openai.Completion.create(prompt=f"<|endoftext|>{text}", **kwargs)
        result = r['choices'][0]
        tokens, logprobs = result["logprobs"]["tokens"][1:], result["logprobs"]["token_logprobs"][1:]

        assert len(tokens) == len(logprobs), f"Expected {len(tokens)} logprobs, got {len(logprobs)}"

        return np.mean(logprobs)
    else:
        with torch.no_grad():
            tokenized = base_tokenizer(text, return_tensors="pt").to(DEVICE)
            labels = tokenized.input_ids
            return -base_model(**tokenized, labels=labels).loss.item()


def get_lls(texts):
    if not args.openai_model:
        return [get_ll(text) for text in texts]
    else:
        global API_TOKEN_COUNTER

        # use GPT2_TOKENIZER to get total number of tokens
        total_tokens = sum(len(GPT2_TOKENIZER.encode(text)) for text in texts)
        API_TOKEN_COUNTER += total_tokens * 2  # multiply by two because OpenAI double-counts echo_prompt tokens

        pool = ThreadPool(args.batch_size)
        return pool.map(get_ll, texts)


# get the average rank of each observed token sorted by model likelihood
def get_rank(text, log=False):
    assert args.openai_model is None, "get_rank not implemented for OpenAI models"

    with torch.no_grad():
        tokenized = base_tokenizer(text, return_tensors="pt").to(DEVICE)
        logits = base_model(**tokenized).logits[:,:-1]
        labels = tokenized.input_ids[:,1:]

        # get rank of each label token in the model's likelihood ordering
        matches = (logits.argsort(-1, descending=True) == labels.unsqueeze(-1)).nonzero()

        assert matches.shape[1] == 3, f"Expected 3 dimensions in matches tensor, got {matches.shape}"

        ranks, timesteps = matches[:,-1], matches[:,-2]

        # make sure we got exactly one match for each timestep in the sequence
        assert (timesteps == torch.arange(len(timesteps)).to(timesteps.device)).all(), "Expected one match per timestep"

        ranks = ranks.float() + 1 # convert to 1-indexed rank
        if log:
            ranks = torch.log(ranks)

        return ranks.float().mean().item()


# get average entropy of each token in the text
def get_entropy(text):
    assert args.openai_model is None, "get_entropy not implemented for OpenAI models"

    with torch.no_grad():
        tokenized = base_tokenizer(text, return_tensors="pt").to(DEVICE)
        logits = base_model(**tokenized).logits[:,:-1]
        neg_entropy = F.softmax(logits, dim=-1) * F.log_softmax(logits, dim=-1)
        return -neg_entropy.sum(-1).mean().item()


def get_roc_metrics(real_preds, machine_preds):
    fpr, tpr, _ = roc_curve([0] * len(real_preds) + [1] * len(machine_preds), real_preds + machine_preds)
    roc_auc = auc(fpr, tpr)
    return fpr.tolist(), tpr.tolist(), float(roc_auc)


def get_precision_recall_metrics(real_preds, machine_preds):
    precision, recall, _ = precision_recall_curve([0] * len(real_preds) + [1] * len(machine_preds), real_preds + machine_preds)
    pr_auc = auc(recall, precision)
    return precision.tolist(), recall.tolist(), float(pr_auc)


# save the ROC curve for each experiment, given a list of output dictionaries, one for each experiment, using colorblind-friendly colors
def save_roc_curves(experiments):
    # first, clear plt
    plt.clf()

    for experiment, color in zip(experiments, COLORS):
        metrics = experiment["metrics"]
        plt.plot(metrics["fpr"], metrics["tpr"], label=f"{experiment['name']}, roc_auc={metrics['roc_auc']:.3f}", color=color)
        # print roc_auc for this experiment
        print(f"{experiment['name']} roc_auc: {metrics['roc_auc']:.3f}")
    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curves ({base_model_name} - {args.mask_filling_model_name})')
    plt.legend(loc="lower right", fontsize=6)
    plt.savefig(f"{SAVE_FOLDER}/roc_curves.png")


# save the histogram of log likelihoods in two side-by-side plots, one for real and real perturbed, and one for machine and machine perturbed
def save_ll_histograms(experiments):
    # first, clear plt
    plt.clf()

    for experiment in experiments:
        try:
            results = experiment["raw_results"]
            # plot histogram of machine/perturbed machine on left, human/perturbed human on right
            plt.figure(figsize=(20, 6))
            plt.subplot(1, 2, 1)
            plt.hist([r["machine_ll"] for r in results], alpha=0.5, bins='auto', label='machine')
            plt.hist([r["perturbed_machine_ll"] for r in results], alpha=0.5, bins='auto', label='perturbed machine')
            plt.xlabel("log likelihood")
            plt.ylabel('count')
            plt.legend(loc='upper right')
            plt.subplot(1, 2, 2)
            plt.hist([r["human_ll"] for r in results], alpha=0.5, bins='auto', label='human')
            plt.hist([r["perturbed_human_ll"] for r in results], alpha=0.5, bins='auto', label='perturbed human')
            plt.xlabel("log likelihood")
            plt.ylabel('count')
            plt.legend(loc='upper right')
            plt.savefig(f"{SAVE_FOLDER}/ll_histograms_{experiment['name']}.png")
        except:
            pass


# save the histograms of log likelihood ratios in two side-by-side plots, one for real and real perturbed, and one for machine and machine perturbed
def save_llr_histograms(experiments):
    # first, clear plt
    plt.clf()

    for experiment in experiments:
        try:
            results = experiment["raw_results"]
            # plot histogram of machine/perturbed machine on left, human/perturbed human on right
            plt.figure(figsize=(20, 6))
            plt.subplot(1, 2, 1)

            # compute the log likelihood ratio for each result
            for r in results:
                r["machine_llr"] = r["machine_ll"] - r["perturbed_machine_ll"]
                r["human_llr"] = r["human_ll"] - r["perturbed_human_ll"]

            plt.hist([r["machine_llr"] for r in results], alpha=0.5, bins='auto', label='machine')
            plt.hist([r["human_llr"] for r in results], alpha=0.5, bins='auto', label='human')
            plt.xlabel("log likelihood ratio")
            plt.ylabel('count')
            plt.legend(loc='upper right')
            plt.savefig(f"{SAVE_FOLDER}/llr_histograms_{experiment['name']}.png")
        except:
            pass


def get_perturbation_results(span_length=10, n_perturbations=1, n_samples=500):
    load_mask_model()

    torch.manual_seed(0)
    np.random.seed(0)

    results = []
    human_text = data["human"]
    machine_text = data["machine"]

    perturb_fn = functools.partial(perturb_texts, span_length=span_length, pct=args.pct_words_masked)

    p_human_text = perturb_fn([x for x in human_text for _ in range(n_perturbations)])
    p_machine_text = perturb_fn([x for x in machine_text for _ in range(n_perturbations)])
    for _ in range(n_perturbation_rounds - 1):
        try:
            p_machine_text, p_human_text = perturb_fn(p_machine_text), perturb_fn(p_human_text)
        except AssertionError:
            break

    assert len(p_machine_text) == len(machine_text) * n_perturbations, f"Expected {len(machine_text) * n_perturbations} perturbed samples, got {len(p_machine_text)}"
    assert len(p_human_text) == len(human_text) * n_perturbations, f"Expected {len(human_text) * n_perturbations} perturbed samples, got {len(p_human_text)}"

    for idx in range(len(human_text)):
        results.append({
            "human": human_text[idx],
            "machine": machine_text[idx],
            "perturbed_machine": p_machine_text[idx * n_perturbations: (idx + 1) * n_perturbations],
            "perturbed_human": p_human_text[idx * n_perturbations: (idx + 1) * n_perturbations]
        })

    load_base_model()

    for res in tqdm.tqdm(results, desc="Computing log likelihoods"):
        p_machine_ll = get_lls(res["perturbed_machine"])
        p_human_ll = get_lls(res["perturbed_human"])
        res["human_ll"] = get_ll(res["human"])
        res["machine_ll"] = get_ll(res["machine"])
        res["all_perturbed_machine_ll"] = p_machine_ll
        res["all_perturbed_human_ll"] = p_human_ll
        res["perturbed_machine_ll"] = np.mean(p_machine_ll)
        res["perturbed_human_ll"] = np.mean(p_human_ll)
        res["perturbed_machine_ll_std"] = np.std(p_machine_ll) if len(p_machine_ll) > 1 else 1
        res["perturbed_human_ll_std"] = np.std(p_human_ll) if len(p_human_ll) > 1 else 1

    return results


def run_perturbation_experiment(results, criterion, span_length=10, n_perturbations=1, n_samples=500):
    # compute diffs with perturbed
    predictions = {'real': [], 'machine': []}
    for res in results:
        if criterion == 'd':
            predictions['real'].append(res['human_ll'] - res['perturbed_human_ll'])
            predictions['machine'].append(res['machine_ll'] - res['perturbed_machine_ll'])
        elif criterion == 'z':
            if res['perturbed_human_ll_std'] == 0:
                res['perturbed_human_ll_std'] = 1
                print("WARNING: std of perturbed human is 0, setting to 1")
                print(f"Number of unique perturbed human texts: {len(set(res['perturbed_human']))}")
                print(f"human text: {res['human']}")
            if res['perturbed_machine_ll_std'] == 0:
                res['perturbed_machine_ll_std'] = 1
                print("WARNING: std of perturbed machine is 0, setting to 1")
                print(f"Number of unique perturbed machine texts: {len(set(res['perturbed_machine']))}")
                print(f"machine text: {res['machine']}")
            predictions['real'].append((res['human_ll'] - res['perturbed_human_ll']) / res['perturbed_human_ll_std'])
            predictions['machine'].append((res['machine_ll'] - res['perturbed_machine_ll']) / res['perturbed_machine_ll_std'])

    print(predictions['real'], predictions['machine'])
    fpr, tpr, roc_auc = get_roc_metrics(predictions['real'], predictions['machine'])
    p, r, pr_auc = get_precision_recall_metrics(predictions['real'], predictions['machine'])
    name = f'perturbation_{n_perturbations}_{criterion}'
    print(f"{name} ROC AUC: {roc_auc}, PR AUC: {pr_auc}")
    return {
        'name': name,
        'predictions': predictions,
        'info': {
            'pct_words_masked': args.pct_words_masked,
            'span_length': span_length,
            'n_perturbations': n_perturbations,
            'n_samples': n_samples,
        },
        'raw_results': results,
        'metrics': {
            'roc_auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
        },
        'pr_metrics': {
            'pr_auc': pr_auc,
            'precision': p,
            'recall': r,
        },
        'loss': 1 - pr_auc,
    }


def run_baseline_threshold_experiment(criterion_fn, name, n_samples=500):
    torch.manual_seed(0)
    np.random.seed(0)
    n_samples = min(len(data["human"]),len(data["machine"]))
    results = []
    for batch in tqdm.tqdm(range(n_samples // batch_size), desc=f"Computing {name} criterion"):
        human_text = data["human"][batch * batch_size:(batch + 1) * batch_size]
        machine_text = data["machine"][batch * batch_size:(batch + 1) * batch_size]

        for idx in range(len(human_text)):
            results.append({
                "human": human_text[idx],
                "human_crit": criterion_fn(human_text[idx]),
                "machine": machine_text[idx],
                "machine_crit": criterion_fn(machine_text[idx]),
            })

    # compute prediction scores for real/machine passages
    predictions = {
        'real': [x["human_crit"] for x in results],
        'machine': [x["machine_crit"] for x in results],
    }

    fpr, tpr, roc_auc = get_roc_metrics(predictions['real'], predictions['machine'])
    p, r, pr_auc = get_precision_recall_metrics(predictions['real'], predictions['machine'])
    print(f"{name}_threshold ROC AUC: {roc_auc}, PR AUC: {pr_auc}")
    return {
        'name': f'{name}_threshold',
        'predictions': predictions,
        'info': {
            'n_samples': n_samples,
        },
        'raw_results': results,
        'metrics': {
            'roc_auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
        },
        'pr_metrics': {
            'pr_auc': pr_auc,
            'precision': p,
            'recall': r,
        },
        'loss': 1 - pr_auc,
    }


# strip newlines from each example; replace one or more newlines with a single space
def strip_newlines(text):
    return ' '.join(text.split())


# trim to shorter length
def trim_to_shorter_length(texta, textb):
    # truncate to shorter of o and s
    shorter_length = min(len(texta.split(' ')), len(textb.split(' ')))
    texta = ' '.join(texta.split(' ')[:shorter_length])
    textb = ' '.join(textb.split(' ')[:shorter_length])
    return texta, textb


def truncate_to_substring(text, substring, idx_occurrence):
    # truncate everything after the idx_occurrence occurrence of substring
    assert idx_occurrence > 0, 'idx_occurrence must be > 0'
    idx = -1
    for _ in range(idx_occurrence):
        idx = text.find(substring, idx + 1)
        if idx == -1:
            return text
    return text[:idx]



def preprocess_text(text):

    text_1 = strip_newlines(text.strip())
    if len(preproc_tokenizer(text_1)["input_ids"]) <= 512:
        return text_1
    else:
        return None

def load_data(dataset, key):

    jsonl_file_path = script_directory+'/SubtaskA/subtaskA_dev_monolingual.jsonl'
    data_all = pd.read_json(jsonl_file_path, lines=True)
    data = {
        "human": [],
        "machine": [],
      }
    for index, row in data_all.iterrows():
        text = preprocess_text(row['text'])
        if text:
            if row['label'] == 0:
                data['human'].append(preprocess_text(row['text']))
            else:
                data['machine'].append(preprocess_text(row['text']))


    # Remove duplicates from each list
    data["human"] = list(set(data["human"]))
    data["machine"] = list(set(data["machine"]))

    return data



def load_base_model_and_tokenizer(name):
    if args.openai_model is None:
        print(f'Loading BASE model {args.base_model_name}...')
        base_model_kwargs = {}
        if 'gpt-j' in name or 'neox' in name:
            base_model_kwargs.update(dict(torch_dtype=torch.float16))
        if 'gpt-j' in name:
            base_model_kwargs.update(dict(revision='float16'))
        base_model = transformers.AutoModelForCausalLM.from_pretrained(name, **base_model_kwargs, cache_dir=cache_dir)
    else:
        base_model = None

    optional_tok_kwargs = {}
    if "facebook/opt-" in name:
        print("Using non-fast tokenizer for OPT")
        optional_tok_kwargs['fast'] = False
    if args.dataset in ['pubmed']:
        optional_tok_kwargs['padding_side'] = 'left'
    base_tokenizer = transformers.AutoTokenizer.from_pretrained(name, **optional_tok_kwargs, cache_dir=cache_dir)
    base_tokenizer.pad_token_id = base_tokenizer.eos_token_id

    return base_model, base_tokenizer


def eval_supervised(data, model):
    print(f'Beginning supervised evaluation with {model}...')
    detector = transformers.AutoModelForSequenceClassification.from_pretrained(model, cache_dir=cache_dir).to(DEVICE)
    tokenizer = transformers.AutoTokenizer.from_pretrained(model, cache_dir=cache_dir)

    real, fake = data['human'], data['machine']
    with torch.no_grad():
        # get predictions for real
        real_preds = []
        for batch in tqdm.tqdm(range(len(real) // batch_size), desc="Evaluating real"):
            batch_real = real[batch * batch_size:(batch + 1) * batch_size]
            batch_real = tokenizer(batch_real, padding=True, truncation=True, max_length=512, return_tensors="pt").to(DEVICE)
            real_preds.extend(detector(**batch_real).logits.softmax(-1)[:,0].tolist())

        # get predictions for fake
        fake_preds = []
        for batch in tqdm.tqdm(range(len(fake) // batch_size), desc="Evaluating fake"):
            batch_fake = fake[batch * batch_size:(batch + 1) * batch_size]
            batch_fake = tokenizer(batch_fake, padding=True, truncation=True, max_length=512, return_tensors="pt").to(DEVICE)
            fake_preds.extend(detector(**batch_fake).logits.softmax(-1)[:,0].tolist())

    predictions = {
        'real': real_preds,
        'machine': fake_preds,
    }

    fpr, tpr, roc_auc = get_roc_metrics(real_preds, fake_preds)
    p, r, pr_auc = get_precision_recall_metrics(real_preds, fake_preds)
    print(f"{model} ROC AUC: {roc_auc}, PR AUC: {pr_auc}")

    # free GPU memory
    del detector
    torch.cuda.empty_cache()

    return {
        'name': model,
        'predictions': predictions,
        'info': {
            'n_samples': n_samples,
        },
        'metrics': {
            'roc_auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
        },
        'pr_metrics': {
            'pr_auc': pr_auc,
            'precision': p,
            'recall': r,
        },
        'loss': 1 - pr_auc,
    }

def check_dataset():

    jsonl_file_path = os.path.join(script_directory, 'SubtaskA', 'subtaskA_dev_monolingual.jsonl')
    if not os.path.isfile(jsonl_file_path):
        print(f"Error: File '{jsonl_file_path}' not found.")
        return False

    # Try to load the JSONL file
    try:
        with open(jsonl_file_path, 'r', encoding='utf-8') as file:
            # Assuming each line in the JSONL file is a valid JSON object
            json_data = [json.loads(line) for line in file]
        print(f"Dataset loaded successfully from '{jsonl_file_path}'.")
        return True
    except Exception as e:
        print(f"Error loading dataset from '{jsonl_file_path}': {str(e)}")
        return False

def download_dataset():

  # Retrieving the data from the public folders
  %cd /content/drive/My Drive/
  !gdown --folder https://drive.google.com/drive/folders/1CAbb3DjrOPBNm0ozVBfhvrEh9P9rAppc



In [ ]:
# Get the directory of the current script
script_directory = r'/content/drive/My Drive/'

if __name__ == '__main__':

    args = DotMap({
        'dataset': "m4",
        'dataset_key': "key",
        'pct_words_masked': 0.3,
        'span_length': 2,
        'n_samples': 200,
        'n_perturbation_list': "1,10",
        'n_perturbation_rounds': 1,
        'base_model_name': "gpt2-medium",
        'scoring_model_name': "",
        'mask_filling_model_name': "t5-large",
        'batch_size': 50,
        'chunk_size': 20,
        'n_similarity_samples': 20,
        'int8': False,
        'half': False,
        'base_half': False,
        'do_top_k': False,
        'top_k': 40,
        'do_top_p': False,
        'top_p': 0.96,
        'output_name': "",
        'openai_model': None,
        'openai_key': None,
        'baselines_only': True,
        'skip_baselines': False,
        'buffer_size': 1,
        'mask_top_p': 1.0,
        'pre_perturb_pct': 0.0,
        'pre_perturb_span_length': 5,
        'random_fills': False,
        'random_fills_tokens': False
    })

    DEVICE = "cuda"

    API_TOKEN_COUNTER = 0



    if not check_dataset():
        download_dataset()

    if args.openai_model is not None:
        import openai
        assert args.openai_key is not None, "Must provide OpenAI API key as --openai_key"
        openai.api_key = args.openai_key

    START_DATE = datetime.datetime.now().strftime('%Y-%m-%d')
    START_TIME = datetime.datetime.now().strftime('%H-%M-%S-%f')

    # define SAVE_FOLDER as the timestamp - base model name - mask filling model name
    # create it if it doesn't exist
    precision_string = "int8" if args.int8 else ("fp16" if args.half else "fp32")
    sampling_string = "top_k" if args.do_top_k else ("top_p" if args.do_top_p else "temp")
    output_subfolder = f"{args.output_name}/" if args.output_name else ""
    if args.openai_model is None:
        base_model_name = args.base_model_name.replace('/', '_')
    else:
        base_model_name = "openai-" + args.openai_model.replace('/', '_')
    scoring_model_string = (f"-{args.scoring_model_name}" if args.scoring_model_name else "").replace('/', '_')
    SAVE_FOLDER = script_directory+f"/tmp_results/{output_subfolder}{base_model_name}{scoring_model_string}-{args.mask_filling_model_name}-{sampling_string}/{START_DATE}-{START_TIME}-{precision_string}-{args.pct_words_masked}-{args.n_perturbation_rounds}-{args.dataset}-{args.n_samples}"
    if not os.path.exists(SAVE_FOLDER):
        os.makedirs(SAVE_FOLDER)
    print(f"Saving results to absolute path: {os.path.abspath(SAVE_FOLDER)}")

    # write args to file
    with open(os.path.join(SAVE_FOLDER, "args.json"), "w") as f:
        json.dump(args.__dict__, f, indent=4)

    mask_filling_model_name = args.mask_filling_model_name
    n_samples = args.n_samples
    batch_size = args.batch_size
    n_perturbation_list = [int(x) for x in args.n_perturbation_list.split(",")]
    n_perturbation_rounds = args.n_perturbation_rounds
    n_similarity_samples = args.n_similarity_samples

    cache_dir = script_directory + '/cache'
    os.environ["XDG_CACHE_HOME"] = cache_dir
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    print(f"Using cache dir {cache_dir}")

    GPT2_TOKENIZER = transformers.GPT2Tokenizer.from_pretrained('gpt2', cache_dir=cache_dir)

    # generic generative model
    base_model, base_tokenizer = load_base_model_and_tokenizer(args.base_model_name)

    # mask filling t5 model
    if not args.baselines_only and not args.random_fills:
        int8_kwargs = {}
        half_kwargs = {}
        if args.int8:
            int8_kwargs = dict(load_in_8bit=True, device_map='auto', torch_dtype=torch.bfloat16)
        elif args.half:
            half_kwargs = dict(torch_dtype=torch.bfloat16)
        print(f'Loading mask filling model {mask_filling_model_name}...')
        mask_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(mask_filling_model_name, **int8_kwargs, **half_kwargs, cache_dir=cache_dir)
        try:
            n_positions = mask_model.config.n_positions
        except AttributeError:
            n_positions = 512
    else:
        n_positions = 512
    preproc_tokenizer = transformers.AutoTokenizer.from_pretrained('t5-small', model_max_length=512, cache_dir=cache_dir)
    mask_tokenizer = transformers.AutoTokenizer.from_pretrained(mask_filling_model_name, model_max_length=n_positions, cache_dir=cache_dir)
    if args.dataset in ['english', 'german']:
        preproc_tokenizer = mask_tokenizer

    load_base_model()

    print(f'Loading dataset {args.dataset}...')
    data = load_data(args.dataset, args.dataset_key)
    if args.random_fills:
        FILL_DICTIONARY = set()
        for texts in data.values():
            for text in texts:
                FILL_DICTIONARY.update(text.split())
        FILL_DICTIONARY = sorted(list(FILL_DICTIONARY))

    if args.scoring_model_name:
        print(f'Loading SCORING model {args.scoring_model_name}...')
        del base_model
        del base_tokenizer
        torch.cuda.empty_cache()
        base_model, base_tokenizer = load_base_model_and_tokenizer(args.scoring_model_name)
        load_base_model()  # Load again because we've deleted/replaced the old model

    # write the data to a json file in the save folder
    with open(os.path.join(SAVE_FOLDER, "raw_data.json"), "w") as f:
        print(f"Writing raw data to {os.path.join(SAVE_FOLDER, 'raw_data.json')}")
        json.dump(data, f)


    outputs = []

    if not args.skip_baselines:
        baseline_outputs = [run_baseline_threshold_experiment(get_ll, "likelihood", n_samples=n_samples)]
        if args.openai_model is None:
            rank_criterion = lambda text: -get_rank(text, log=False)
            baseline_outputs.append(run_baseline_threshold_experiment(rank_criterion, "rank", n_samples=n_samples))
            logrank_criterion = lambda text: -get_rank(text, log=True)
            baseline_outputs.append(run_baseline_threshold_experiment(logrank_criterion, "log_rank", n_samples=n_samples))
            entropy_criterion = lambda text: get_entropy(text)
            baseline_outputs.append(run_baseline_threshold_experiment(entropy_criterion, "entropy", n_samples=n_samples))

        baseline_outputs.append(eval_supervised(data, model='roberta-base-openai-detector'))
        baseline_outputs.append(eval_supervised(data, model='roberta-large-openai-detector'))

        # write likelihood threshold results to a file
        with open(os.path.join(SAVE_FOLDER, f"likelihood_threshold_results.json"), "w") as f:
            json.dump(baseline_outputs[0], f)

        if args.openai_model is None:
            # write rank threshold results to a file
            with open(os.path.join(SAVE_FOLDER, f"rank_threshold_results.json"), "w") as f:
                json.dump(baseline_outputs[1], f)

            # write log rank threshold results to a file
            with open(os.path.join(SAVE_FOLDER, f"logrank_threshold_results.json"), "w") as f:
                json.dump(baseline_outputs[2], f)

            # write entropy threshold results to a file
            with open(os.path.join(SAVE_FOLDER, f"entropy_threshold_results.json"), "w") as f:
                json.dump(baseline_outputs[3], f)

        # write supervised results to a file
        with open(os.path.join(SAVE_FOLDER, f"roberta-base-openai-detector_results.json"), "w") as f:
            json.dump(baseline_outputs[-2], f)

        # write supervised results to a file
        with open(os.path.join(SAVE_FOLDER, f"roberta-large-openai-detector_results.json"), "w") as f:
            json.dump(baseline_outputs[-1], f)

        outputs += baseline_outputs

    if not args.baselines_only:
        # run perturbation experiments
        for n_perturbations in n_perturbation_list:
            perturbation_results = get_perturbation_results(args.span_length, n_perturbations, n_samples)
            for perturbation_mode in ['d', 'z']:
                output = run_perturbation_experiment(
                    perturbation_results, perturbation_mode, span_length=args.span_length, n_perturbations=n_perturbations, n_samples=n_samples)
                outputs.append(output)
                with open(os.path.join(SAVE_FOLDER, f"perturbation_{n_perturbations}_{perturbation_mode}_results.json"), "w") as f:
                    json.dump(output, f)

    save_roc_curves(outputs)
    save_ll_histograms(outputs)
    save_llr_histograms(outputs)

    # move results folder from tmp_results/ to results/, making sure necessary directories exist
    new_folder = SAVE_FOLDER.replace("tmp_results", "results")
    if not os.path.exists(os.path.dirname(new_folder)):
        os.makedirs(os.path.dirname(new_folder))
    os.rename(SAVE_FOLDER, new_folder)

    print(f"Used an *estimated* {API_TOKEN_COUNTER} API tokens (may be inaccurate)")

In [ ]:
def get_roc_metrics(predictions_dict):
    real_preds = predictions_dict['real']
    machine_preds = predictions_dict['machine']
    fpr, tpr, _ = roc_curve([0] * len(real_preds) + [1] * len(machine_preds), real_preds + machine_preds)
    roc_auc = auc(fpr, tpr)
    return fpr.tolist(), tpr.tolist(), float(roc_auc)

def get_precision_recall_metrics(predictions_dict):
    real_preds = predictions_dict['real']
    machine_preds = predictions_dict['machine']
    precision, recall, _ = precision_recall_curve([0] * len(real_preds) + [1] * len(machine_preds), real_preds + machine_preds)
    pr_auc = auc(recall, precision)
    return precision.tolist(), recall.tolist(), float(pr_auc)

def get_accuracy_f1_metrics(predictions_dict):

    real_preds = predictions_dict['real']
    machine_preds = predictions_dict['machine']
    preds = real_preds + machine_preds
    precision, recall, thresholds = precision_recall_curve([0] * len(real_preds) + [1] * len(machine_preds),preds )

    # Calculate F1 score for each threshold
    f1_scores = [f1_score([0] * len(real_preds) + [1] * len(machine_preds), [1 if score >= threshold else 0 for score in preds]) for threshold in thresholds]

    # Find the index of the maximum F1 score
    optimal_f1_threshold_index = f1_scores.index(max(f1_scores))
    # Get the corresponding threshold
    optimal_f1_threshold = thresholds[optimal_f1_threshold_index]
    optimal_f1_score = f1_scores[optimal_f1_threshold_index]

    # Calculate accuracy for each threshold
    accuracies = [accuracy_score([0] * len(real_preds) + [1] * len(machine_preds), [1 if score >= threshold else 0 for score in preds]) for threshold in thresholds]

    # Find the index of the maximum accuracy
    optimal_accuracy_threshold_index = accuracies.index(max(accuracies))

    # Get the corresponding threshold and accuracy
    optimal_accuracy_threshold = thresholds[optimal_accuracy_threshold_index]
    optimal_accuracy = accuracies[optimal_accuracy_threshold_index]


    optimal_precision = precision_score([0] * len(real_preds) + [1] * len(machine_preds), [1 if score >= optimal_f1_threshold else 0 for score in preds])
    optimal_recall = recall_score([0] * len(real_preds) + [1] * len(machine_preds), [1 if score >= optimal_f1_threshold else 0 for score in preds])




    return optimal_accuracy, optimal_precision, optimal_recall, optimal_f1_score, optimal_accuracy_threshold, optimal_f1_threshold

def plot_confusion_matrix(predictions_dict):


    real_preds = predictions_dict['real']
    machine_preds = predictions_dict['machine']

    preds = real_preds + machine_preds
    precision, recall, thresholds = precision_recall_curve([0] * len(real_preds) + [1] * len(machine_preds),preds )

    # Calculate F1 score for each threshold
    f1_scores = [f1_score([0] * len(real_preds) + [1] * len(machine_preds), [1 if score >= threshold else 0 for score in preds]) for threshold in thresholds]

    # Find the index of the maximum F1 score
    optimal_f1_threshold_index = f1_scores.index(max(f1_scores))
    # Get the corresponding threshold
    optimal_f1_threshold = thresholds[optimal_f1_threshold_index]

    # Apply the threshold to get binary predictions
    binary_predictions = (preds > optimal_f1_threshold).astype(int)

    # Compute the confusion matrix
    cm = confusion_matrix([0] * len(real_preds) + [1] * len(machine_preds), binary_predictions)

    # Plot the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Human', 'Machine'], yticklabels=['Human', 'Machine'])
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix')
    plt.show()

def create_evaluation_df(data):

  metric_data = {}
  for key in data.keys():
    metric_data[key] = {}
    metric_data[key]['accuracy'], metric_data[key]['precision'], metric_data[key]['recall'], metric_data[key]['f1'],  _, _ = get_accuracy_f1_metrics(data[key]['predictions'])
    metric_data[key]['roc_auc'] = data[key]['metrics']['roc_auc']

  return pd.DataFrame.from_dict(metric_data, orient='index')

def print_results(SAVE_FOLDER):

  # Define the file names
  file_names = [
      "rank_threshold_results.json",
      "logrank_threshold_results.json",
      "entropy_threshold_results.json",
      "roberta-base-openai-detector_results.json",
      "roberta-large-openai-detector_results.json",
      "likelihood_threshold_results.json",
      "perturbation_1_d_results.json",
      "perturbation_1_z_results.json",
      "perturbation_10_d_results.json",
      "perturbation_10_z_results.json"
  ]

  # Create a dictionary to store the data
  data = {}

  # Read each file and assign the data to variables
  for file_name in file_names:
      file_path = os.path.join(SAVE_FOLDER, file_name)

      # Check if the file exists
      if os.path.exists(file_path):
          with open(file_path, "r") as f:
              data[file_name.replace('_results.json', '')] = json.load(f)
      else:
          print(f"The file {file_path} does not exist.")
  return data, create_evaluation_df(data)

In [ ]:
data, evaluation_df = print_results('/content/drive/MyDrive/results/gpt2-medium-t5-large-temp/2023-12-07-00-04-08-643443-fp32-0.3-1-m4-200')

In [ ]:
evaluation_df

,accuracy,precision,recall,f1,roc_auc
rank_threshold,0.797500,0.761261,0.845000,0.800948,0.851950
logrank_threshold,0.732500,0.676806,0.890000,0.768898,0.813325
entropy_threshold,0.502500,0.501253,1.000000,0.667780,0.197225
roberta-base-openai-detector,0.620506,0.620471,0.999592,0.765671,0.489529
roberta-large-openai-detector,0.624051,0.621381,0.998776,0.766124,0.461347
likelihood_threshold,0.705000,0.636691,0.885000,0.740586,0.760750
perturbation_1_d,0.575131,0.506406,0.985564,0.669042,0.597746
perturbation_1_z,0.575131,0.506406,0.985564,0.669042,0.597746
perturbation_10_d,0.661745,0.573799,0.862205,0.689040,0.701595
perturbation_10_z,0.645013,0.555160,0.921260,0.692820,0.682782


In [ ]:
plot_confusion_matrix(data['rank_threshold']['predictions'])

In [ ]:
plot_confusion_matrix(data['perturbation_10_d']['predictions'])